In [20]:
%%writefile lab/vector.cpp

#include <CL/sycl.hpp>
using namespace sycl;

static const size_t N = 100;

#include <chrono>
#include <vector>
#include <numeric>
#include <algorithm>

void work(queue &q) {
  std::cout << "Device : "
            << q.get_device().get_info<info::device::name>()
            << std::endl;
            
  std::vector<int> vector1(N);
  std::iota(vector1.begin(), vector1.end(), 1);

  std::cout << "vector1  : ";
  for (const auto& elem: vector1) std::cout << elem << ' ';
  std::cout << std::endl;

  std::vector<int> vector2(N);
  std::generate(vector2.begin(), vector2.end(), [] {
      static int i = 0;
      return (++i * N);
  });  

  std::cout << "vector2  : ";
  for (const auto& elem: vector2) std::cout << elem << ' ';
  std::cout << std::endl;


  buffer vector1_buffer {vector1};
  buffer vector2_buffer {vector2};
    

  q.submit([&](handler &h) {
    auto vector1_accessor = vector1_buffer.get_access<access::mode::read_write>(h);
    auto vector2_accessor = vector2_buffer.get_access <access::mode::read >(h);

    h.parallel_for<class test>(range<1>(N), [=](id<1> index) {

      vector1_accessor[index] += vector2_accessor[index];
    });
  });
  q.wait();
  vector1_buffer.get_access<access::mode::read>();

  std::cout << "vector1 + vector2  : ";
  for (const auto& elem: vector1) std::cout << elem << ' ';
  std::cout << std::endl;
}

int main() {  
  try {
    queue q;
    auto start = std::chrono::high_resolution_clock::now();
    work(q);
    auto elapsed = std::chrono::high_resolution_clock::now() - start;
    long long microseconds = std::chrono::duration_cast<std::chrono::microseconds>(elapsed).count();
    std::cout << "Elapsed microseconds : " << microseconds << std::endl;
  } catch (exception e) {
    std::cerr << "Exception: " << e.what() << std::endl;
    std::terminate();
  } catch (...) {
    std::cerr << "Unknown exception" << std::endl;
    std::terminate();
  }
}

Overwriting lab/vector.cpp


In [21]:
! chmod 755 q; chmod 755 run_vector.sh; if [ -x "$(command -v qsub)" ]; then ./q run_vector.sh; else ./run_vector.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2296154.v-qsvr-1           ...ub-singleuser u192698         00:01:10 R jupyterhub     
2296208.v-qsvr-1           run_vector.sh    u192698                0 Q batch          

Waiting for Output ████████████████████ Done⬇

########################################################################
#      Date:           Tue 16 May 2023 05:06:14 PM PDT
#    Job ID:           2296208.v-qsvr-1.aidevcloud
#      User:           u192698
# Resources:           cput=75:00:00,neednodes=1:gpu:ppn=2,nodes=1:gpu:ppn=2,walltime=06:00:00
########################################################################

## u192698 is compiling SYCL_Essentials Module1 -- oneAPI Intro sample - 1 of 1 vector.cpp
Device : Intel(R) UHD Graphics [0x9a60]
vector1  : 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 1